**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

**NOTE:**

FILTER = (c+5)*3

max_batches = c*2000 (unless it's smaller than 6000)

steps = %80, %90*max_batches


In [ ]:
!cp cfg/yolov3-spp.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=32/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@width=416@width=416@' cfg/yolov3_training.cfg
!sed -i '776 s@height=416@height=416@' cfg/yolov3_training.cfg

In [ ]:
!printf "car\n" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/weights/car_test/car_test_416' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!mkdir /content/images/

import glob, os
zip_list = glob.glob("/content/drive/MyDrive/dataset/car_test/*.zip")
rar_list = glob.glob("/content/drive/MyDrive/dataset/car_test/*.rar")

for zip in zip_list:
  print(zip, " zip")
  os.system("unzip {} -d /content/images/".format(zip))

for rar in rar_list:
  print(rar, " rar")
  os.system("unrar e {} /content/images/".format(rar))

In [ ]:
import glob
images_list = glob.glob("/content/images/*.jpg")

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 



**6) Start the training**

darknet53.conv.74

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show